# Trade Sentiment Analysis

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load the datasets
historical_data = pd.read_csv('/content/historical_data.csv')
fear_greed_data = pd.read_csv('/content/fear_greed_index.csv')

# Convert the timestamp columns to datetime format
historical_data['Timestamp IST'] = pd.to_datetime(historical_data['Timestamp IST'], format='%d-%m-%Y %H:%M')
fear_greed_data['timestamp'] = pd.to_datetime(fear_greed_data['timestamp'], unit='s')

# Create a new column in historical_data with just the date part
historical_data['Date'] = historical_data['Timestamp IST'].dt.date
fear_greed_data['Date'] = fear_greed_data['timestamp'].dt.date

# Merge the datasets on the 'Date' column
merged_data = pd.merge(historical_data, fear_greed_data, on='Date', how='left')

merged_data.columns

Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp', 'Date', 'timestamp', 'value', 'classification', 'date'],
      dtype='object')

In [5]:
# Data preprocessing
# Remove rows with missing PnL or sentiment data
df = merged_data.dropna(subset=['Closed PnL', 'value']).copy()

# Convert Date to datetime if not already
df['Date'] = pd.to_datetime(df['Date'])

# Create sentiment categories
def categorize_sentiment(value):
   if value <= 25:
       return 'Extreme Fear'
   elif value <= 45:
       return 'Fear'
   elif value <= 55:
       return 'Neutral'
   elif value <= 75:
       return 'Greed'
   else:
       return 'Extreme Greed'

df['sentiment_category'] = df['value'].apply(categorize_sentiment)

# Create position direction mapping
df['position_type'] = df['Side'].map({'BUY': 'Long', 'SELL': 'Short'})

print("Data shape:", df.shape)
print("Date range:", df['Date'].min(), "to", df['Date'].max())
print("\nSentiment distribution:")
print(df['sentiment_category'].value_counts())

Data shape: (4653, 23)
Date range: 2024-09-20 00:00:00 to 2025-04-25 00:00:00

Sentiment distribution:
sentiment_category
Fear             1574
Neutral           901
Extreme Fear      898
Greed             721
Extreme Greed     559
Name: count, dtype: int64


In [7]:
def correlation_analysis(df):
    if 'value' not in df.columns or 'Closed PnL' not in df.columns:
        return None, None

    analysis_df = df[['value', 'Closed PnL']].dropna()
    if analysis_df.empty:
        return None, None

    pearson_corr, _ = pearsonr(analysis_df['value'], analysis_df['Closed PnL'])
    spearman_corr, _ = spearmanr(analysis_df['value'], analysis_df['Closed PnL'])

    # 1. Scatter Plot with Trend Line
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=df['value'],
        y=df['Closed PnL'],
        mode='markers',
        marker=dict(
            color=df['value'],
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="Fear & Greed Index"),
            size=8,
            opacity=0.7
        ),
        text=df.get('sentiment_category', 'N/A'),
        hovertemplate='<b>Sentiment:</b> %{text}<br><b>Index:</b> %{x}<br><b>P&L:</b> %{y:.2f}<extra></extra>',
        name='Trades'
    ))

    if len(analysis_df) > 1:
        z = np.polyfit(analysis_df['value'], analysis_df['Closed PnL'], 1)
        p = np.poly1d(z)
        x_trend = np.linspace(analysis_df['value'].min(), analysis_df['value'].max(), 100)
        fig1.add_trace(go.Scatter(
            x=x_trend,
            y=p(x_trend),
            mode='lines',
            line=dict(color='red', dash='dash', width=2),
            name=f'Trend Line (R={pearson_corr:.3f})'
        ))

    fig1.update_layout(
        title=f"Market Sentiment vs Trading Performance<br>Pearson R={pearson_corr:.3f}, Spearman R={spearman_corr:.3f}",
        xaxis_title="Fear & Greed Index",
        yaxis_title="P&L",
        height=600,
        showlegend=True
    )
    fig1.show()

    # 2. Distribution Plots
    fig2 = make_subplots(rows=1, cols=2, subplot_titles=('Sentiment Distribution', 'P&L Distribution'))

    fig2.add_trace(go.Histogram(x=df['value'], nbinsx=20, marker_color='lightblue'), row=1, col=1)
    fig2.add_trace(go.Histogram(x=df['Closed PnL'], nbinsx=30, marker_color='lightgreen'), row=1, col=2)

    fig2.update_layout(title_text="Data Distributions", height=400, showlegend=False)
    fig2.show()

    # 3. Correlation Heatmap (Optional)
    numeric_cols = ['value', 'Closed PnL']
    for col in ['Execution Price', 'Size USD']:
        if col in df.columns:
            numeric_cols.append(col)

    if len(numeric_cols) >= 2:
        corr_data = df[numeric_cols].corr()
        fig3 = go.Figure(data=go.Heatmap(
            z=corr_data.values,
            x=corr_data.columns,
            y=corr_data.columns,
            colorscale='RdBu',
            zmid=0,
            text=corr_data.round(3).values,
            texttemplate="%{text}",
            textfont={"size": 12}
        ))
        fig3.update_layout(title="Correlation Matrix", height=500)
        fig3.show()

    return pearson_corr, spearman_corr
pearson_corr, spearman_corr = correlation_analysis(df)

In [11]:
def sentiment_shift_analysis(df):
    df_sorted = df.sort_values('Date').copy()
    df_sorted['sentiment_change'] = df_sorted['value'].diff()

    def categorize_shift(change):
        if pd.isna(change): return 'No Change'
        elif change > 10: return 'Major Increase'
        elif change > 5: return 'Increase'
        elif change < -10: return 'Major Decrease'
        elif change < -5: return 'Decrease'
        else: return 'Stable'

    df_sorted['sentiment_shift'] = df_sorted['sentiment_change'].apply(categorize_shift)
    df_sorted = df_sorted.dropna(subset=['sentiment_change'])

    if df_sorted.empty:
        return None

    shift_perf = df_sorted.groupby('sentiment_shift').agg({
        'Closed PnL': ['mean', 'median', 'std', 'count'],
        'value': 'mean'
    }).round(4)

    shift_perf.columns = ['Avg_PnL', 'Median_PnL', 'PnL_Std', 'Trade_Count', 'Avg_Sentiment']
    shift_perf = shift_perf.reset_index()

    # 1. Box Plot
    fig1 = go.Figure()
    colors = ['red', 'orange', 'gray', 'lightgreen', 'darkgreen']
    for i, shift_type in enumerate(shift_perf['sentiment_shift']):
        shift_data = df_sorted[df_sorted['sentiment_shift'] == shift_type]
        if not shift_data.empty:
            fig1.add_trace(go.Box(
                y=shift_data['Closed PnL'],
                name=shift_type,
                boxpoints='outliers',
                marker_color=colors[i % len(colors)]
            ))
    fig1.update_layout(
        title="P&L Distribution by Sentiment Shift Type",
        xaxis_title="Sentiment Shift Type",
        yaxis_title="P&L",
        height=500
    )
    fig1.show()

    # 2. Time Series with Major Shifts
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=df_sorted['Date'],
        y=df_sorted['value'],
        mode='lines',
        name='Fear & Greed Index',
        line=dict(color='blue', width=2)
    ))

    major_shifts = df_sorted[df_sorted['sentiment_shift'].isin(['Major Increase', 'Major Decrease'])]
    if not major_shifts.empty:
        fig2.add_trace(go.Scatter(
            x=major_shifts['Date'],
            y=major_shifts['value'],
            mode='markers',
            name='Major Shifts',
            marker=dict(
                size=12,
                color=['red' if x == 'Major Decrease' else 'green' for x in major_shifts['sentiment_shift']],
                symbol='diamond',
                line=dict(width=2, color='white')
            ),
            text=major_shifts['sentiment_shift'],
            hovertemplate='<b>Date:</b> %{x}<br><b>Sentiment:</b> %{y}<br><b>Shift:</b> %{text}<extra></extra>'
        ))

    fig2.update_layout(
        title="Sentiment Over Time with Major Shift Markers",
        xaxis_title="Date",
        yaxis_title="Fear & Greed Index",
        height=500
    )
    fig2.show()

    # 3. Performance Metrics Subplots
    fig3 = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Average P&L', 'P&L Volatility', 'Trade Count')
    )

    fig3.add_trace(go.Bar(
        x=shift_perf['sentiment_shift'],
        y=shift_perf['Avg_PnL'],
        marker_color=['red' if x < 0 else 'green' for x in shift_perf['Avg_PnL']],
        text=[f"{x:.2f}" for x in shift_perf['Avg_PnL']],
        textposition='outside'
    ), row=1, col=1)

    fig3.add_trace(go.Bar(
        x=shift_perf['sentiment_shift'],
        y=shift_perf['PnL_Std'],
        marker_color='orange',
        text=[f"{x:.2f}" for x in shift_perf['PnL_Std']],
        textposition='outside'
    ), row=1, col=2)

    fig3.add_trace(go.Bar(
        x=shift_perf['sentiment_shift'],
        y=shift_perf['Trade_Count'],
        marker_color='lightblue',
        text=shift_perf['Trade_Count'].astype(str),
        textposition='outside'
    ), row=1, col=3)

    fig3.update_layout(
        title_text="Performance Metrics by Sentiment Shift Type",
        height=500,
        showlegend=False
    )
    fig3.update_xaxes(tickangle=45)
    fig3.show()

    return shift_perf
shift_perf = sentiment_shift_analysis(df)


In [13]:
def volatility_analysis(df):
    required_cols = ['sentiment_category', 'Closed PnL', 'value', 'Date']
    if not all(col in df.columns for col in required_cols):
        return None

    df_clean = df.dropna(subset=['sentiment_category', 'Closed PnL'])
    if df_clean.empty:
        return None

    def percentile_range(x):
        return x.quantile(0.95) - x.quantile(0.05) if len(x) > 1 else 0

    vol_stats = df_clean.groupby('sentiment_category').agg({
        'Closed PnL': ['std', 'var', 'mean', 'count', percentile_range],
        'value': 'mean'
    }).round(4)

    vol_stats.columns = ['PnL_Std', 'PnL_Variance', 'Avg_PnL', 'Trade_Count', 'PnL_Range_90', 'Avg_Sentiment']
    vol_stats = vol_stats.reset_index()

    vol_stats['Risk_Adj_Return'] = vol_stats.apply(
        lambda row: row['Avg_PnL'] / row['PnL_Std'] if row['PnL_Std'] > 0 else 0, axis=1
    )

    # 1. Volatility Bar Chart
    sorted_vol = vol_stats.sort_values('PnL_Std')
    fig1 = go.Figure()
    fig1.add_trace(go.Bar(
        x=sorted_vol['sentiment_category'],
        y=sorted_vol['PnL_Std'],
        marker_color=['red' if x > sorted_vol['PnL_Std'].mean() else 'orange' for x in sorted_vol['PnL_Std']],
        text=[f"{x:.2f}" for x in sorted_vol['PnL_Std']],
        textposition='outside'
    ))
    fig1.update_layout(
        title="P&L Volatility by Market Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Standard Deviation of P&L",
        height=500
    )
    fig1.show()

    # 2. Risk-Adjusted Return Chart
    sorted_rar = vol_stats.sort_values('Risk_Adj_Return')
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(
        x=sorted_rar['sentiment_category'],
        y=sorted_rar['Risk_Adj_Return'],
        marker_color=['green' if x > 0 else 'red' for x in sorted_rar['Risk_Adj_Return']],
        text=[f"{x:.3f}" for x in sorted_rar['Risk_Adj_Return']],
        textposition='outside'
    ))
    fig2.update_layout(
        title="Risk-Adjusted Return by Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Return / Volatility",
        height=500
    )
    fig2.show()

    # 3. P&L Distribution Box Plots
    fig3 = go.Figure()
    colors = ['red', 'orange', 'gray', 'lightgreen', 'darkgreen']
    for i, sentiment in enumerate(df_clean['sentiment_category'].unique()):
        fig3.add_trace(go.Box(
            y=df_clean[df_clean['sentiment_category'] == sentiment]['Closed PnL'],
            name=sentiment,
            marker_color=colors[i % len(colors)],
            boxpoints='outliers'
        ))
    fig3.update_layout(
        title="P&L Distribution by Sentiment Category",
        xaxis_title="Sentiment Category",
        yaxis_title="Closed PnL",
        height=500
    )
    fig3.show()

    # 4. Rolling Volatility Over Time
    df_sorted = df_clean.sort_values('Date').copy()
    fig4 = go.Figure()
    for window in [10, 20, 30]:
        if len(df_sorted) >= window:
            df_sorted[f'Rolling_Vol_{window}'] = df_sorted['Closed PnL'].rolling(window, min_periods=window//2).std()
            fig4.add_trace(go.Scatter(
                x=df_sorted['Date'],
                y=df_sorted[f'Rolling_Vol_{window}'],
                mode='lines',
                name=f'{window}-Day Rolling Volatility'
            ))

    fig4.add_trace(go.Scatter(
        x=df_sorted['Date'],
        y=[0]*len(df_sorted),
        mode='markers',
        marker=dict(
            size=8,
            color=df_sorted['value'],
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="Fear & Greed Index", x=1.25)
        ),
        name='Sentiment Level',
        yaxis='y2'
    ))

    fig4.update_layout(
        title="Rolling Volatility Over Time with Sentiment Overlay",
        xaxis_title="Date",
        yaxis_title="Rolling Std Dev",
        height=500,
        yaxis2=dict(overlaying='y', side='right', showgrid=False)
    )
    fig4.show()

    return vol_stats
volatility_stats = volatility_analysis(df)

In [14]:
def performance_consistency_analysis(df):
    required_cols = ['sentiment_category', 'Closed PnL']
    if not all(col in df.columns for col in required_cols):
        return None

    df_clean = df.dropna(subset=['sentiment_category', 'Closed PnL'])
    if df_clean.empty:
        return None

    def win_rate(x):
        return (x > 0).sum() / len(x) if len(x) > 0 else 0

    def safe_percentile(x, p): return np.percentile(x, p) if len(x) > 1 else np.nan
    def safe_skew(x): return stats.skew(x) if len(x) > 2 else np.nan

    stats_df = df_clean.groupby('sentiment_category').agg({
        'Closed PnL': [
            'mean', 'median', 'std', 'count',
            win_rate,
            lambda x: safe_percentile(x, 25),
            lambda x: safe_percentile(x, 75),
            safe_skew
        ]
    }).round(4)

    stats_df.columns = ['Mean_PnL', 'Median_PnL', 'Std_PnL', 'Trade_Count',
                        'Win_Rate', 'Q1_PnL', 'Q3_PnL', 'Skewness']
    stats_df = stats_df.reset_index()

    stats_df['CV'] = stats_df.apply(
        lambda row: abs(row['Std_PnL'] / row['Mean_PnL']) if row['Mean_PnL'] != 0 else np.inf, axis=1
    )
    stats_df['Consistency_Score'] = stats_df.apply(
        lambda row: row['Win_Rate'] / row['CV'] if row['CV'] > 0 and row['CV'] != np.inf else 0, axis=1
    )

    stats_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # VISUALIZATION
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Win Rate (%)', 'Consistency Score',
                        'Mean vs Median P&L', 'Trade Count Distribution'],
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "pie"}]]
    )

    # 1. Win Rate
    fig.add_trace(go.Bar(
        x=stats_df['sentiment_category'],
        y=stats_df['Win_Rate'] * 100,
        text=[f"{x:.1f}%" for x in (stats_df['Win_Rate'] * 100)],
        marker_color='green',
        textposition='outside'
    ), row=1, col=1)

    # 2. Consistency Score
    fig.add_trace(go.Bar(
        x=stats_df['sentiment_category'],
        y=stats_df['Consistency_Score'].fillna(0),
        text=[f"{x:.2f}" for x in stats_df['Consistency_Score'].fillna(0)],
        marker_color='blue',
        textposition='outside'
    ), row=1, col=2)

    # 3. Mean vs Median P&L
    fig.add_trace(go.Bar(
        x=stats_df['sentiment_category'],
        y=stats_df['Mean_PnL'],
        name='Mean P&L',
        marker_color='lightblue',
        offsetgroup=1
    ), row=2, col=1)

    fig.add_trace(go.Bar(
        x=stats_df['sentiment_category'],
        y=stats_df['Median_PnL'],
        name='Median P&L',
        marker_color='darkblue',
        offsetgroup=2
    ), row=2, col=1)

    # 4. Pie chart
    fig.add_trace(go.Pie(
        labels=stats_df['sentiment_category'],
        values=stats_df['Trade_Count'],
        name='Trade Distribution',
        showlegend=False
    ), row=2, col=2)

    fig.update_layout(
        title_text="Performance Consistency Across Sentiment Categories",
        height=800,
        showlegend=True
    )

    fig.update_xaxes(title_text="Sentiment Category", row=1, col=1)
    fig.update_yaxes(title_text="Win Rate (%)", row=1, col=1)
    fig.update_xaxes(title_text="Sentiment Category", row=1, col=2)
    fig.update_yaxes(title_text="Consistency Score", row=1, col=2)
    fig.update_xaxes(title_text="Sentiment Category", row=2, col=1)
    fig.update_yaxes(title_text="P&L Value", row=2, col=1)

    fig.show()
    return stats_df
consistency_stats = performance_consistency_analysis(df)


In [16]:
def sentiment_duration_analysis(df):
    df = df.dropna(subset=['Date', 'sentiment_category', 'Closed PnL']).copy()
    df['Date'] = pd.to_datetime(df['Date'])

    df = df.sort_values('Date')
    df['sentiment_group'] = (df['sentiment_category'] != df['sentiment_category'].shift()).cumsum()

    duration_stats = df.groupby(['sentiment_group', 'sentiment_category']).agg({
        'Date': ['min', 'max'],
        'Closed PnL': ['mean', 'sum'],
        'value': 'mean'
    }).reset_index()

    duration_stats.columns = ['Group', 'Sentiment', 'Start_Date', 'End_Date', 'Avg_PnL', 'Total_PnL', 'Avg_Sentiment']
    duration_stats['Actual_Duration_Days'] = (duration_stats['End_Date'] - duration_stats['Start_Date']).dt.days + 1

    duration_stats['Duration_Category'] = pd.cut(
        duration_stats['Actual_Duration_Days'],
        bins=[0, 3, 7, 14, 30, float('inf')],
        labels=['1-3 days', '4-7 days', '8-14 days', '15-30 days', '30+ days'],
        include_lowest=True
    ).fillna('1-3 days')

    duration_performance = duration_stats.groupby(['Sentiment', 'Duration_Category']).agg({
        'Avg_PnL': 'mean',
        'Total_PnL': 'sum',
        'Actual_Duration_Days': 'mean',
        'Group': 'count'
    }).reset_index().rename(columns={
        'Group': 'Period_Count'
    })

    # Visualization
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            'Performance by Duration Category',
            'Sentiment Duration Distribution',
            'Total P&L by Duration & Sentiment',
            'Average P&L Heatmap'
        ],
        specs=[[{"type": "bar"}, {"type": "histogram"}],
               [{"type": "bar"}, {"type": "heatmap"}]]
    )

    colors = ['red', 'orange', 'gray', 'lightgreen', 'darkgreen']

    # Subplot 1: Avg PnL bar (Performance by Duration Category)
    for i, sentiment in enumerate(duration_performance['Sentiment'].unique()):
      sub = duration_performance[duration_performance['Sentiment'] == sentiment]

      sub = sub.dropna(subset=['Avg_PnL'])

      fig.add_trace(
          go.Bar(
              x=sub['Duration_Category'],
              y=sub['Avg_PnL'],
              name=sentiment,
              marker_color=colors[i % len(colors)],
              text=[f"{x:.2f}" for x in sub['Avg_PnL']],
              textposition='outside'
          ),
          row=1, col=1
      )


    # Subplot 2: Duration Histogram
    fig.add_trace(
        go.Histogram(
            x=duration_stats['Actual_Duration_Days'],
            nbinsx=20,
            marker_color='orange',
            showlegend=False
        ),
        row=1, col=2
    )

    # Subplot 3: Total PnL bar
    pivot_total = duration_performance.pivot(index='Duration_Category', columns='Sentiment', values='Total_PnL').fillna(0)
    for i, sentiment in enumerate(pivot_total.columns):
        fig.add_trace(
            go.Bar(
                x=pivot_total.index,
                y=pivot_total[sentiment],
                name=sentiment,
                marker_color=colors[i % len(colors)],
                opacity=0.7
            ),
            row=2, col=1
        )

    # Subplot 4: Heatmap
    pivot_avg = duration_performance.pivot(index='Duration_Category', columns='Sentiment', values='Avg_PnL').fillna(0)
    fig.add_trace(
        go.Heatmap(
            z=pivot_avg.values,
            x=pivot_avg.columns,
            y=pivot_avg.index,
            colorscale='RdYlGn',
            zmid=0,
            text=pivot_avg.round(2).values,
            texttemplate="%{text}",
            textfont={"size": 10},
            showscale=True,
            colorbar=dict(x=1.1)
        ),
        row=2, col=2
    )

    fig.update_layout(
        title_text="Sentiment Duration Impact on Trading Performance",
        height=800
    )

    fig.update_xaxes(title_text="Duration Category", row=1, col=1)
    fig.update_yaxes(title_text="Average P&L", row=1, col=1)

    fig.update_xaxes(title_text="Duration (Days)", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)

    fig.update_xaxes(title_text="Duration Category", row=2, col=1)
    fig.update_yaxes(title_text="Total P&L", row=2, col=1)

    fig.show()

    return duration_stats, duration_performance
duration_stats, duration_performance = sentiment_duration_analysis(df)


In [17]:
def long_short_analysis(df):
    # Check required columns
    required_cols = ['sentiment_category', 'position_type', 'Closed PnL', 'value']
    if not all(col in df.columns for col in required_cols):
        return None

    # Clean data
    df_filtered = df.dropna(subset=['position_type', 'sentiment_category', 'Closed PnL']).copy()

    # Group statistics
    position_performance = df_filtered.groupby(['sentiment_category', 'position_type']).agg({
        'Closed PnL': ['mean', 'median', 'sum', 'std', 'count'],
        'value': 'mean'
    }).round(4)

    position_performance.columns = ['Avg_PnL', 'Median_PnL', 'Total_PnL', 'PnL_Std', 'Trade_Count', 'Avg_Sentiment']
    position_performance = position_performance.reset_index()
    position_performance['PnL_Std'] = position_performance['PnL_Std'].fillna(0)

    # Win rate calculation
    win_rates = df_filtered.groupby(['sentiment_category', 'position_type'])['Closed PnL'] \
        .apply(lambda x: (x > 0).sum() / len(x)).reset_index(name='Win_Rate')

    position_performance = position_performance.merge(win_rates, on=['sentiment_category', 'position_type'])

    # Detect long and short types
    pos_types = position_performance['position_type'].unique()
    long_pos = [p for p in pos_types if 'long' in str(p).lower() or 'buy' in str(p).lower()]
    short_pos = [p for p in pos_types if 'short' in str(p).lower() or 'sell' in str(p).lower()]

    if not long_pos: long_pos = [pos_types[0]]
    if not short_pos and len(pos_types) > 1: short_pos = [pos_types[1]]

    long_data = position_performance[position_performance['position_type'].isin(long_pos)]
    short_data = position_performance[position_performance['position_type'].isin(short_pos)]

    # Plotting
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Average P&L: Long vs Short', 'Win Rate Comparison',
                        'Total P&L by Strategy', 'Risk-Return Profile'],
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )

    # 1. Avg PnL
    fig.add_trace(go.Bar(x=long_data['sentiment_category'], y=long_data['Avg_PnL'],
                         name='Long', marker_color='green', text=long_data['Avg_PnL'].round(2),
                         textposition='outside'), row=1, col=1)
    fig.add_trace(go.Bar(x=short_data['sentiment_category'], y=short_data['Avg_PnL'],
                         name='Short', marker_color='red', text=short_data['Avg_PnL'].round(2),
                         textposition='outside'), row=1, col=1)

    # 2. Win Rate
    fig.add_trace(go.Bar(x=long_data['sentiment_category'], y=long_data['Win_Rate'] * 100,
                         name='Long Win Rate', marker_color='lightgreen',
                         text=(long_data['Win_Rate'] * 100).round(1), textposition='outside'),
                  row=1, col=2)
    fig.add_trace(go.Bar(x=short_data['sentiment_category'], y=short_data['Win_Rate'] * 100,
                         name='Short Win Rate', marker_color='lightcoral',
                         text=(short_data['Win_Rate'] * 100).round(1), textposition='outside'),
                  row=1, col=2)

    # 3. Total PnL
    fig.add_trace(go.Bar(x=long_data['sentiment_category'], y=long_data['Total_PnL'],
                         name='Long Total P&L', marker_color='darkgreen',
                         text=long_data['Total_PnL'].round(0), textposition='outside'),
                  row=2, col=1)
    fig.add_trace(go.Bar(x=short_data['sentiment_category'], y=short_data['Total_PnL'],
                         name='Short Total P&L', marker_color='darkred',
                         text=short_data['Total_PnL'].round(0), textposition='outside'),
                  row=2, col=1)

    # 4. Risk vs Return
    fig.add_trace(go.Scatter(x=long_data['PnL_Std'], y=long_data['Avg_PnL'],
                             mode='markers+text', text=long_data['sentiment_category'],
                             marker=dict(color='green', size=12), name='Long'),
                  row=2, col=2)
    fig.add_trace(go.Scatter(x=short_data['PnL_Std'], y=short_data['Avg_PnL'],
                             mode='markers+text', text=short_data['sentiment_category'],
                             marker=dict(color='red', size=12), name='Short'),
                  row=2, col=2)

    fig.update_layout(title_text="Long vs Short Trades Performance Across Sentiment Levels",
                      height=800, showlegend=True)

    fig.update_xaxes(title_text="Sentiment", row=1, col=1)
    fig.update_yaxes(title_text="Avg P&L", row=1, col=1)

    fig.update_xaxes(title_text="Sentiment", row=1, col=2)
    fig.update_yaxes(title_text="Win Rate (%)", row=1, col=2)

    fig.update_xaxes(title_text="Sentiment", row=2, col=1)
    fig.update_yaxes(title_text="Total P&L", row=2, col=1)

    fig.update_xaxes(title_text="PnL Std Dev", row=2, col=2)
    fig.update_yaxes(title_text="Avg P&L", row=2, col=2)

    fig.show()
    return position_performance
position_performance = long_short_analysis(df)


In [19]:
def risk_reward_analysis(df):
    """
    Calculate risk-reward ratios and create visualizations
    """
    def calculate_risk_reward_metrics(group):
        positive_trades = group[group['Closed PnL'] > 0]['Closed PnL']
        negative_trades = group[group['Closed PnL'] < 0]['Closed PnL']

        avg_win = positive_trades.mean() if len(positive_trades) > 0 else 0
        avg_loss = abs(negative_trades.mean()) if len(negative_trades) > 0 else 0

        risk_reward_ratio = avg_win / avg_loss if avg_loss != 0 else np.inf
        win_rate = len(positive_trades) / len(group) if len(group) > 0 else 0
        profit_factor = positive_trades.sum() / abs(negative_trades.sum()) if negative_trades.sum() != 0 else np.inf
        expectancy = group['Closed PnL'].mean()

        return pd.Series({
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'risk_reward_ratio': risk_reward_ratio,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'expectancy': expectancy,
            'total_trades': len(group),
            'total_pnl': group['Closed PnL'].sum()
        })

    # Calculate metrics
    risk_reward_stats = df.groupby('sentiment_category').apply(calculate_risk_reward_metrics).round(4)
    risk_reward_stats = risk_reward_stats.reset_index()

    # Handle infinite values for visualization
    risk_reward_display = risk_reward_stats.copy()
    risk_reward_display['risk_reward_ratio'] = risk_reward_display['risk_reward_ratio'].replace([np.inf, -np.inf], 999)
    risk_reward_display['profit_factor'] = risk_reward_display['profit_factor'].replace([np.inf, -np.inf], 999)

    # Create visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Risk-Reward Ratio by Sentiment',
            'Profit Factor Analysis',
            'Win Rate vs Risk-Reward',
            'Expected Value (Expectancy)'
        ),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "scatter"}, {"type": "bar"}]]
    )

    # Colors
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

    # 1. Risk-Reward Ratio
    fig.add_trace(
        go.Bar(
            x=risk_reward_display['sentiment_category'],
            y=risk_reward_display['risk_reward_ratio'],
            name='Risk-Reward Ratio',
            marker_color=colors[:len(risk_reward_display)],
            text=[f"{x:.2f}" if x < 999 else "∞" for x in risk_reward_display['risk_reward_ratio']],
            textposition='outside'
        ),
        row=1, col=1
    )

    # 2. Profit Factor
    fig.add_trace(
        go.Bar(
            x=risk_reward_display['sentiment_category'],
            y=risk_reward_display['profit_factor'],
            name='Profit Factor',
            marker_color=colors[:len(risk_reward_display)],
            text=[f"{x:.2f}" if x < 999 else "∞" for x in risk_reward_display['profit_factor']],
            textposition='outside',
            showlegend=False
        ),
        row=1, col=2
    )

    # 3. Win Rate vs Risk-Reward Scatter
    fig.add_trace(
        go.Scatter(
            x=risk_reward_display['win_rate'] * 100,
            y=risk_reward_display['risk_reward_ratio'],
            mode='markers+text',
            text=risk_reward_display['sentiment_category'],
            textposition='top center',
            name='Win Rate vs Risk-Reward',
            marker=dict(
                size=15,
                color=colors[:len(risk_reward_display)],
                line=dict(width=2, color='white')
            ),
            showlegend=False
        ),
        row=2, col=1
    )

    # 4. Expectancy
    expectancy_colors = ['#2ca02c' if x > 0 else '#d62728' for x in risk_reward_display['expectancy']]
    fig.add_trace(
        go.Bar(
            x=risk_reward_display['sentiment_category'],
            y=risk_reward_display['expectancy'],
            name='Expectancy',
            marker_color=expectancy_colors,
            text=[f"{x:.2f}" for x in risk_reward_display['expectancy']],
            textposition='outside',
            showlegend=False
        ),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        title_text="Risk-Reward Analysis Across Sentiment Periods",
        height=800,
        showlegend=True,
        font=dict(size=12)
    )

    # Update axis labels
    fig.update_xaxes(title_text="Sentiment Category", row=1, col=1)
    fig.update_yaxes(title_text="Risk-Reward Ratio", row=1, col=1)

    fig.update_xaxes(title_text="Sentiment Category", row=1, col=2)
    fig.update_yaxes(title_text="Profit Factor", row=1, col=2)

    fig.update_xaxes(title_text="Win Rate (%)", row=2, col=1)
    fig.update_yaxes(title_text="Risk-Reward Ratio", row=2, col=1)

    fig.update_xaxes(title_text="Sentiment Category", row=2, col=2)
    fig.update_yaxes(title_text="Expectancy", row=2, col=2)

    fig.show()

    return risk_reward_stats

def create_individual_risk_reward_charts(df):
    """
    Create individual charts for risk-reward analysis
    """
    # Calculate metrics
    def calculate_metrics(group):
        positive_trades = group[group['Closed PnL'] > 0]['Closed PnL']
        negative_trades = group[group['Closed PnL'] < 0]['Closed PnL']

        avg_win = positive_trades.mean() if len(positive_trades) > 0 else 0
        avg_loss = abs(negative_trades.mean()) if len(negative_trades) > 0 else 0

        return pd.Series({
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'risk_reward_ratio': avg_win / avg_loss if avg_loss != 0 else np.inf,
            'win_rate': len(positive_trades) / len(group) if len(group) > 0 else 0,
            'profit_factor': positive_trades.sum() / abs(negative_trades.sum()) if negative_trades.sum() != 0 else np.inf,
            'expectancy': group['Closed PnL'].mean(),
            'total_pnl': group['Closed PnL'].sum()
        })

    stats = df.groupby('sentiment_category').apply(calculate_metrics).round(4)
    stats = stats.reset_index()

    # Chart 1: Risk-Reward Ratio
    fig1 = go.Figure()
    rr_values = stats['risk_reward_ratio'].replace([np.inf, -np.inf], 999)
    fig1.add_trace(go.Bar(
        x=stats['sentiment_category'],
        y=rr_values,
        marker_color='steelblue',
        text=[f"{x:.2f}" if x < 999 else "∞" for x in rr_values],
        textposition='outside'
    ))
    fig1.update_layout(
        title="Risk-Reward Ratio by Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Risk-Reward Ratio",
        height=500
    )
    fig1.show()

    # Chart 2: Win Rate
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(
        x=stats['sentiment_category'],
        y=stats['win_rate'] * 100,
        marker_color='lightgreen',
        text=[f"{x:.1f}%" for x in (stats['win_rate'] * 100)],
        textposition='outside'
    ))
    fig2.update_layout(
        title="Win Rate by Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Win Rate (%)",
        height=500
    )
    fig2.show()

    # Chart 3: Expectancy
    fig3 = go.Figure()
    colors = ['green' if x > 0 else 'red' for x in stats['expectancy']]
    fig3.add_trace(go.Bar(
        x=stats['sentiment_category'],
        y=stats['expectancy'],
        marker_color=colors,
        text=[f"{x:.2f}" for x in stats['expectancy']],
        textposition='outside'
    ))
    fig3.update_layout(
        title="Expected Value (Expectancy) by Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Expectancy",
        height=500
    )
    fig3.show()

    # Chart 4: Profit Factor
    fig4 = go.Figure()
    pf_values = stats['profit_factor'].replace([np.inf, -np.inf], 999)
    fig4.add_trace(go.Bar(
        x=stats['sentiment_category'],
        y=pf_values,
        marker_color='orange',
        text=[f"{x:.2f}" if x < 999 else "∞" for x in pf_values],
        textposition='outside'
    ))
    fig4.update_layout(
        title="Profit Factor by Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Profit Factor",
        height=500
    )
    fig4.show()

    # Chart 5: Average Win vs Average Loss
    fig5 = go.Figure()
    fig5.add_trace(go.Bar(
        x=stats['sentiment_category'],
        y=stats['avg_win'],
        name='Average Win',
        marker_color='green',
        offsetgroup=1
    ))
    fig5.add_trace(go.Bar(
        x=stats['sentiment_category'],
        y=stats['avg_loss'],
        name='Average Loss',
        marker_color='red',
        offsetgroup=2
    ))
    fig5.update_layout(
        title="Average Win vs Average Loss by Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Amount",
        height=500,
        barmode='group'
    )
    fig5.show()

    # Chart 6: Total P&L
    fig6 = go.Figure()
    colors = ['green' if x > 0 else 'red' for x in stats['total_pnl']]
    fig6.add_trace(go.Bar(
        x=stats['sentiment_category'],
        y=stats['total_pnl'],
        marker_color=colors,
        text=[f"{x:.0f}" for x in stats['total_pnl']],
        textposition='outside'
    ))
    fig6.update_layout(
        title="Total P&L by Sentiment",
        xaxis_title="Sentiment Category",
        yaxis_title="Total P&L",
        height=500
    )
    fig6.show()

    return stats


risk_reward_stats = risk_reward_analysis(df)
individual_stats = create_individual_risk_reward_charts(df)

In [26]:
def asset_specific_sentiment_analysis(df):
    """
    Comprehensive analysis of how different assets perform under varying sentiment conditions
    """
    # Get top assets by trade count
    top_assets = df['Coin'].value_counts().head(12).index.tolist()
    df_assets = df[df['Coin'].isin(top_assets)].copy()

    # Calculate comprehensive performance metrics by asset and sentiment
    def calculate_asset_metrics(group):
        positive_trades = group[group['Closed PnL'] > 0]['Closed PnL']
        negative_trades = group[group['Closed PnL'] < 0]['Closed PnL']

        avg_win = positive_trades.mean() if len(positive_trades) > 0 else 0
        avg_loss = abs(negative_trades.mean()) if len(negative_trades) > 0 else 0

        return pd.Series({
            'avg_pnl': group['Closed PnL'].mean(),
            'total_pnl': group['Closed PnL'].sum(),
            'pnl_std': group['Closed PnL'].std(),
            'trade_count': len(group),
            'avg_sentiment': group['value'].mean(),
            'win_rate': len(positive_trades) / len(group) if len(group) > 0 else 0,
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'risk_reward_ratio': avg_win / avg_loss if avg_loss != 0 else np.inf,
            'profit_factor': positive_trades.sum() / abs(negative_trades.sum()) if negative_trades.sum() != 0 else np.inf,
            'expectancy': group['Closed PnL'].mean(),
            'sharpe_ratio': group['Closed PnL'].mean() / group['Closed PnL'].std() if group['Closed PnL'].std() != 0 else 0
        })

    asset_performance = df_assets.groupby(['Coin', 'sentiment_category']).apply(calculate_asset_metrics).round(4)
    asset_performance = asset_performance.reset_index()

    # Calculate sentiment sensitivity (correlation between sentiment and performance for each asset)
    sentiment_sensitivity = []
    for coin in top_assets:
        coin_data = df_assets[df_assets['Coin'] == coin]
        if len(coin_data) > 10:  # Minimum data points for reliable correlation
            try:
                corr, p_value = pearsonr(coin_data['value'], coin_data['Closed PnL'])
                sentiment_sensitivity.append({
                    'Coin': coin,
                    'Sentiment_Correlation': corr,
                    'P_Value': p_value,
                    'Trade_Count': len(coin_data),
                    'Total_PnL': coin_data['Closed PnL'].sum(),
                    'Avg_PnL': coin_data['Closed PnL'].mean(),
                    'Win_Rate': (coin_data['Closed PnL'] > 0).sum() / len(coin_data)
                })
            except:
                # Handle cases where correlation cannot be calculated
                sentiment_sensitivity.append({
                    'Coin': coin,
                    'Sentiment_Correlation': 0,
                    'P_Value': 1,
                    'Trade_Count': len(coin_data),
                    'Total_PnL': coin_data['Closed PnL'].sum(),
                    'Avg_PnL': coin_data['Closed PnL'].mean(),
                    'Win_Rate': (coin_data['Closed PnL'] > 0).sum() / len(coin_data)
                })

    sensitivity_df = pd.DataFrame(sentiment_sensitivity).round(4)

    # Calculate best and worst performing sentiment periods for each asset
    asset_sentiment_ranking = []
    for coin in top_assets:
        coin_performance = asset_performance[asset_performance['Coin'] == coin]
        if len(coin_performance) > 0:
            best_sentiment = coin_performance.loc[coin_performance['avg_pnl'].idxmax(), 'sentiment_category']
            worst_sentiment = coin_performance.loc[coin_performance['avg_pnl'].idxmin(), 'sentiment_category']
            best_pnl = coin_performance['avg_pnl'].max()
            worst_pnl = coin_performance['avg_pnl'].min()

            asset_sentiment_ranking.append({
                'Coin': coin,
                'Best_Sentiment': best_sentiment,
                'Best_Avg_PnL': best_pnl,
                'Worst_Sentiment': worst_sentiment,
                'Worst_Avg_PnL': worst_pnl,
                'Sentiment_Range': best_pnl - worst_pnl
            })

    ranking_df = pd.DataFrame(asset_sentiment_ranking).round(4)

    print("=== ASSET-SPECIFIC SENTIMENT ANALYSIS ===")
    print("\n1. Top Assets Performance by Sentiment:")
    print(asset_performance.head(15))
    print("\n2. Sentiment Sensitivity Rankings:")
    print(sensitivity_df.sort_values('Sentiment_Correlation', ascending=False))
    print("\n3. Best/Worst Sentiment Periods by Asset:")
    print(ranking_df.sort_values('Sentiment_Range', ascending=False))

    return asset_performance, sensitivity_df, ranking_df

def create_asset_sentiment_visualizations(df, asset_performance, sensitivity_df, ranking_df):
    """
    Create comprehensive visualizations for asset-specific sentiment analysis
    """
    # Get top assets for visualization
    top_assets = df['Coin'].value_counts().head(8).index.tolist()

    # Create comprehensive subplot visualization
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Average P&L by Asset & Sentiment',
            'Sentiment Sensitivity (Correlation)',
            'Asset Performance Heatmap (Avg P&L)',
            'Win Rate by Asset & Sentiment',
            'Risk-Reward Ratio by Asset',
            'Total P&L by Asset & Sentiment'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"type": "heatmap"}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Colors for different sentiment categories
    sentiment_colors = {
        'Very Negative': '#d62728',
        'Negative': '#ff7f0e',
        'Neutral': '#2ca02c',
        'Positive': '#1f77b4',
        'Very Positive': '#9467bd'
    }

    # 1. Average P&L by asset and sentiment (grouped bar chart)
    sentiments = asset_performance['sentiment_category'].unique()
    assets_to_show = top_assets[:6]  # Show top 6 assets for clarity

    for sentiment in sentiments:
        sentiment_data = asset_performance[
            (asset_performance['sentiment_category'] == sentiment) &
            (asset_performance['Coin'].isin(assets_to_show))
        ]
        if len(sentiment_data) > 0:
            fig.add_trace(
                go.Bar(
                    x=sentiment_data['Coin'],
                    y=sentiment_data['avg_pnl'],
                    name=sentiment,
                    marker_color=sentiment_colors.get(sentiment, '#8c564b'),
                    text=sentiment_data['avg_pnl'].round(2),
                    textposition='outside',
                    legendgroup='sentiment'
                ),
                row=1, col=1
            )

    # 2. Sentiment sensitivity
    sensitivity_sorted = sensitivity_df.sort_values('Sentiment_Correlation', ascending=True)
    colors_sens = ['red' if x < 0 else 'green' for x in sensitivity_sorted['Sentiment_Correlation']]

    fig.add_trace(
        go.Bar(
            x=sensitivity_sorted['Sentiment_Correlation'],
            y=sensitivity_sorted['Coin'],
            orientation='h',
            name='Sentiment Correlation',
            marker_color=colors_sens,
            text=sensitivity_sorted['Sentiment_Correlation'].round(3),
            textposition='outside',
            showlegend=False
        ),
        row=1, col=2
    )

    # 3. Performance heatmap
    pivot_performance = asset_performance[asset_performance['Coin'].isin(assets_to_show)].pivot(
        index='Coin', columns='sentiment_category', values='avg_pnl'
    ).fillna(0)

    if not pivot_performance.empty:
        fig.add_trace(
            go.Heatmap(
                z=pivot_performance.values,
                x=pivot_performance.columns,
                y=pivot_performance.index,
                colorscale='RdYlGn',
                zmid=0,
                text=pivot_performance.round(2).values,
                texttemplate="%{text}",
                textfont={"size": 10},
                showscale=True,
                name='Performance Heatmap'
            ),
            row=2, col=1
        )

    # 4. Win rate by asset and sentiment
    for sentiment in sentiments:
        sentiment_data = asset_performance[
            (asset_performance['sentiment_category'] == sentiment) &
            (asset_performance['Coin'].isin(assets_to_show))
        ]
        if len(sentiment_data) > 0:
            fig.add_trace(
                go.Scatter(
                    x=sentiment_data['Coin'],
                    y=sentiment_data['win_rate'] * 100,
                    mode='markers+lines',
                    name=f'{sentiment} Win Rate',
                    marker=dict(size=8, color=sentiment_colors.get(sentiment, '#8c564b')),
                    line=dict(color=sentiment_colors.get(sentiment, '#8c564b')),
                    legendgroup='winrate',
                    showlegend=False
                ),
                row=2, col=2
            )

    # 5. Risk-Reward Ratio by asset (average across sentiments)
    rr_by_asset = asset_performance.groupby('Coin')['risk_reward_ratio'].mean().reset_index()
    rr_by_asset = rr_by_asset[rr_by_asset['Coin'].isin(assets_to_show)]
    rr_by_asset['risk_reward_ratio'] = rr_by_asset['risk_reward_ratio'].replace([np.inf, -np.inf], 999)

    fig.add_trace(
        go.Bar(
            x=rr_by_asset['Coin'],
            y=rr_by_asset['risk_reward_ratio'],
            name='Avg Risk-Reward',
            marker_color='steelblue',
            text=[f"{x:.2f}" if x < 999 else "∞" for x in rr_by_asset['risk_reward_ratio']],
            textposition='outside',
            showlegend=False
        ),
        row=3, col=1
    )

    # 6. Total P&L by asset and sentiment
    for sentiment in sentiments:
        sentiment_data = asset_performance[
            (asset_performance['sentiment_category'] == sentiment) &
            (asset_performance['Coin'].isin(assets_to_show))
        ]
        if len(sentiment_data) > 0:
            fig.add_trace(
                go.Bar(
                    x=sentiment_data['Coin'],
                    y=sentiment_data['total_pnl'],
                    name=f'{sentiment} Total P&L',
                    marker_color=sentiment_colors.get(sentiment, '#8c564b'),
                    text=sentiment_data['total_pnl'].round(0),
                    textposition='outside',
                    legendgroup='totalpnl',
                    showlegend=False
                ),
                row=3, col=2
            )

    # Update layout
    fig.update_layout(
        title_text="Asset-Specific Sentiment Analysis Dashboard",
        height=1200,
        showlegend=True,
        font=dict(size=11)
    )

    # Update axis labels
    fig.update_xaxes(title_text="Asset", row=1, col=1)
    fig.update_yaxes(title_text="Average P&L", row=1, col=1)

    fig.update_xaxes(title_text="Correlation Coefficient", row=1, col=2)
    fig.update_yaxes(title_text="Asset", row=1, col=2)

    fig.update_xaxes(title_text="Sentiment Category", row=2, col=1)
    fig.update_yaxes(title_text="Asset", row=2, col=1)

    fig.update_xaxes(title_text="Asset", row=2, col=2)
    fig.update_yaxes(title_text="Win Rate (%)", row=2, col=2)

    fig.update_xaxes(title_text="Asset", row=3, col=1)
    fig.update_yaxes(title_text="Risk-Reward Ratio", row=3, col=1)

    fig.update_xaxes(title_text="Asset", row=3, col=2)
    fig.update_yaxes(title_text="Total P&L", row=3, col=2)

    fig.show()

    return fig

def create_individual_asset_charts(df, asset_performance, sensitivity_df):
    """
    Create individual detailed charts for asset-specific analysis
    """
    top_assets = df['Coin'].value_counts().head(6).index.tolist()

    # Chart 1: Sentiment Sensitivity Ranking
    fig1 = go.Figure()
    sensitivity_sorted = sensitivity_df.sort_values('Sentiment_Correlation', ascending=False)
    colors = ['darkgreen' if x > 0.1 else 'lightgreen' if x > 0 else 'lightcoral' if x > -0.1 else 'darkred'
              for x in sensitivity_sorted['Sentiment_Correlation']]

    fig1.add_trace(go.Bar(
        x=sensitivity_sorted['Coin'],
        y=sensitivity_sorted['Sentiment_Correlation'],
        marker_color=colors,
        text=sensitivity_sorted['Sentiment_Correlation'].round(3),
        textposition='outside'
    ))
    fig1.update_layout(
        title="Sentiment Sensitivity by Asset (Correlation with P&L)",
        xaxis_title="Asset",
        yaxis_title="Correlation Coefficient",
        height=500
    )
    fig1.show()

    # Chart 2: Asset Performance Comparison
    fig2 = go.Figure()
    avg_performance = asset_performance.groupby('Coin').agg({
        'avg_pnl': 'mean',
        'win_rate': 'mean',
        'trade_count': 'sum'
    }).reset_index()
    avg_performance = avg_performance[avg_performance['Coin'].isin(top_assets)]

    colors = ['green' if x > 0 else 'red' for x in avg_performance['avg_pnl']]
    fig2.add_trace(go.Bar(
        x=avg_performance['Coin'],
        y=avg_performance['avg_pnl'],
        marker_color=colors,
        text=avg_performance['avg_pnl'].round(2),
        textposition='outside'
    ))
    fig2.update_layout(
        title="Average P&L by Asset (All Sentiment Periods)",
        xaxis_title="Asset",
        yaxis_title="Average P&L",
        height=500
    )
    fig2.show()

    # Chart 3: Win Rate vs Sentiment Sensitivity
    fig3 = go.Figure()
    merged_data = sensitivity_df.merge(
        avg_performance[['Coin', 'win_rate']], on='Coin', how='inner'
    )

    fig3.add_trace(go.Scatter(
        x=merged_data['Sentiment_Correlation'],
        y=merged_data['win_rate'] * 100,
        mode='markers+text',
        text=merged_data['Coin'],
        textposition='top center',
        marker=dict(
            size=12,
            color=merged_data['Sentiment_Correlation'],
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="Sentiment Correlation")
        )
    ))
    fig3.update_layout(
        title="Win Rate vs Sentiment Sensitivity",
        xaxis_title="Sentiment Correlation",
        yaxis_title="Win Rate (%)",
        height=500
    )
    fig3.show()

    # Chart 4: Detailed Asset Breakdown
    fig4 = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Total P&L by Asset', 'Trade Count by Asset'),
        vertical_spacing=0.15
    )

    # Total P&L
    total_pnl = asset_performance.groupby('Coin')['total_pnl'].sum().reset_index()
    total_pnl = total_pnl[total_pnl['Coin'].isin(top_assets)]
    colors = ['green' if x > 0 else 'red' for x in total_pnl['total_pnl']]

    fig4.add_trace(
        go.Bar(
            x=total_pnl['Coin'],
            y=total_pnl['total_pnl'],
            marker_color=colors,
            text=total_pnl['total_pnl'].round(0),
            textposition='outside',
            name='Total P&L'
        ),
        row=1, col=1
    )

    # Trade Count
    trade_counts = asset_performance.groupby('Coin')['trade_count'].sum().reset_index()
    trade_counts = trade_counts[trade_counts['Coin'].isin(top_assets)]

    fig4.add_trace(
        go.Bar(
            x=trade_counts['Coin'],
            y=trade_counts['trade_count'],
            marker_color='steelblue',
            text=trade_counts['trade_count'],
            textposition='outside',
            name='Trade Count'
        ),
        row=2, col=1
    )

    fig4.update_layout(
        title_text="Asset Trading Activity Overview",
        height=700,
        showlegend=False
    )

    fig4.update_xaxes(title_text="Asset", row=1, col=1)
    fig4.update_yaxes(title_text="Total P&L", row=1, col=1)
    fig4.update_xaxes(title_text="Asset", row=2, col=1)
    fig4.update_yaxes(title_text="Number of Trades", row=2, col=1)

    fig4.show()

    return fig1, fig2, fig3, fig4

# Usage example:
asset_performance, sensitivity_df, ranking_df = asset_specific_sentiment_analysis(df)
comprehensive_fig = create_asset_sentiment_visualizations(df, asset_performance, sensitivity_df, ranking_df)
individual_charts = create_individual_asset_charts(df, asset_performance, sensitivity_df)

=== ASSET-SPECIFIC SENTIMENT ANALYSIS ===

1. Top Assets Performance by Sentiment:
     Coin sentiment_category   avg_pnl    total_pnl    pnl_std  trade_count  \
0    @107      Extreme Greed   48.6460   12793.9074   232.7820        263.0   
1    @107              Greed    0.0000       0.0000     0.0000         80.0   
2    @107            Neutral  257.2739  194499.0725   704.4792        756.0   
3    AAVE       Extreme Fear    0.0000       0.0000     0.0000         12.0   
4    AAVE      Extreme Greed  100.0597    8705.1911   233.7152         87.0   
5    AAVE               Fear  844.3333    2533.0000  1195.6813          3.0   
6   AIXBT       Extreme Fear    0.0000       0.0000     0.0000         18.0   
7   AIXBT               Fear  105.8135    9734.8407   294.0720         92.0   
8   AIXBT              Greed  284.8385   38168.3549   848.4680        134.0   
9   AIXBT            Neutral    0.0000       0.0000        NaN          1.0   
10    BTC       Extreme Fear -152.2155  -42620.3

# Some more insights

In [ ]:
# Directional Analysis

# Long vs Short Performance
avg_pnl_by_side_sentiment = merged_data.groupby(['Side', 'classification'])['Closed PnL'].mean().reset_index()
fig_long_short = px.bar(avg_pnl_by_side_sentiment, x='Side', y='Closed PnL', color='classification',
                        barmode='group', title='Long vs Short Performance',
                        labels={'side': 'Trade Side', 'Closed PnL': 'Average P&L'})
fig_long_short.show()

# Side-by-Side Comparison
fig_side_by_side = px.box(merged_data, x='classification', y='Closed PnL', color='Side',
                          title='Side-by-Side Comparison',
                          labels={'classification': 'Market Sentiment', 'Closed PnL': 'P&L'})
fig_side_by_side.show()



In [ ]:
# Calculate average P&L by market sentiment classification
avg_pnl = merged_data.groupby('classification')['Closed PnL'].mean().reset_index()

# Define the order for sentiment classification
sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']

# Create a bar chart showing average P&L by market sentiment
fig1 = px.bar(avg_pnl, x='classification', y='Closed PnL',
              title='Average P&L by Market Sentiment',
              labels={'Closed PnL': 'Average P&L', 'classification': 'Market Sentiment'},
              color='Closed PnL',
              color_continuous_scale=['red', 'lightgrey', 'green'],
              category_orders={'classification': sentiment_order})

# Show the plot
fig1.show()


In [ ]:
# Win Rate by Sentiment
win_rate = merged_data.groupby('classification').apply(lambda x: (x['Closed PnL'] > 0).mean() * 100).reset_index()
win_rate.columns = ['classification', 'Win Rate (%)']

# Define the order for sentiment classification
sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']

fig2 = px.bar(win_rate, x='classification', y='Win Rate (%)',
              title='Win Rate by Market Sentiment',
              labels={'Win Rate (%)': 'Win Rate (%)', 'classification': 'Market Sentiment'},
              color='Win Rate (%)',
              color_continuous_scale=['lightgrey', 'darkgreen'],
              category_orders={'classification': sentiment_order})
fig2.show()

# Volume vs Performance Scatter
daily_stats = merged_data.groupby(['Date', 'classification']).agg(
    trade_count=('Account', 'count'),
    total_pnl=('Closed PnL', 'sum')
).reset_index()

fig3 = px.scatter(daily_stats, x='trade_count', y='total_pnl',
                  color='classification', size='trade_count',
                  title='Daily Trade Volume vs Performance by Market Sentiment',
                  labels={'trade_count': 'Number of Trades',
                          'total_pnl': 'Total Daily P&L',
                          'classification': 'Market Sentiment'},
                  hover_data=['Date'],
                  category_orders={'classification': sentiment_order})
fig3.show()


In [ ]:
# Calculate daily P&L and get daily sentiment data
daily_pnl = merged_data.groupby('Date')['Closed PnL'].sum().reset_index()
daily_sentiment = merged_data.groupby('Date')['classification'].first().reset_index()

# Create sentiment numeric mapping
sentiment_map = {
    'Extreme Fear': 0,
    'Fear': 25,
    'Neutral': 50,
    'Greed': 75,
    'Extreme Greed': 100
}

# Convert sentiment to numeric values
daily_sentiment['Sentiment_Value'] = daily_sentiment['classification'].apply(lambda x: sentiment_map.get(x, 50))

# Merge the daily P&L and sentiment data
timeline_df = pd.merge(daily_pnl, daily_sentiment, on='Date')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add P&L bars
fig.add_trace(
    go.Bar(
        x=timeline_df['Date'],
        y=timeline_df['Closed PnL'],
        name="Daily P&L",
        marker_color=['red' if x < 0 else 'green' for x in timeline_df['Closed PnL']]
    ),
    secondary_y=False,
)

# Add sentiment line
fig.add_trace(
    go.Scatter(
        x=timeline_df['Date'],
        y=timeline_df['Sentiment_Value'],
        name="Market Sentiment",
        line=dict(color='blue', width=2),
        mode='lines+markers'
    ),
    secondary_y=True,
)

# Update layout for interactivity
fig.update_layout(
    title="Daily P&L vs Market Sentiment Over Time",
    xaxis_title="Date",
    yaxis_title="Daily P&L",
    height=600,
    width=1000,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode='x unified'
)

# Set y-axes titles
fig.update_yaxes(title_text="Daily P&L", secondary_y=False)
fig.update_yaxes(
    title_text="Market Sentiment",
    secondary_y=True,
    tickvals=list(sentiment_map.values()),
    ticktext=list(sentiment_map.keys())
)

# Show the plot
fig.show()
